# **Importar librerías**

In [1]:
import pandas as pd
import numpy as np
import requests

## **01. Lectura de base de datos: Necesario para conocer el orden de los NIT**

In [2]:
NITS = pd.read_csv('data_sedes_00.csv')
NITS['label']

0       890907797
1       890928257
2       890904138
3       890913902
4       890984423
          ...    
140     890984002
141     890980040
142     890901389
143    8909029226
144     811043174
Name: label, Length: 145, dtype: int64

In [6]:
url = f"https://webservices.metropol.gov.co/simapi/api/Terceros/PmesTercero?Doc={800180330}"

    # Realizar la solicitud al web service
response = requests.get(url)
data = response.json()
data

{'Empresa': 'COMPAÑIA DE ALIMENTOS COLOMBIANOS CALCO S.A',
 'NIT': 800180330,
 'IdTercero': 224618,
 'CantidadColaboradores': 255,
 'CantidadEncuestasCumplidas': 1018,
 'CantInstalaciones': 24,
 'CantRutas': 0,
 'Instalaciones': [{'IdInstalacion': 127140,
   'NombreInstalcion': 'R01 Poblado',
   'CoordenadaX': -75.565847,
   'CoordenadaY': 6.209256,
   'CantEmpleados': 32,
   'CantidadEncuestasCumplidas': 30},
  {'IdInstalacion': 127163,
   'NombreInstalcion': 'R02 Unicentro',
   'CoordenadaX': -75.5874153,
   'CoordenadaY': 6.2411471,
   'CantEmpleados': 0,
   'CantidadEncuestasCumplidas': 34},
  {'IdInstalacion': 127164,
   'NombreInstalcion': 'R03 Campestre',
   'CoordenadaX': -75.56922639999999,
   'CoordenadaY': 6.185952899999999,
   'CantEmpleados': 0,
   'CantidadEncuestasCumplidas': 39},
  {'IdInstalacion': 127165,
   'NombreInstalcion': 'R04 Tesoro',
   'CoordenadaX': -75.5588321,
   'CoordenadaY': 6.1976243,
   'CantEmpleados': 0,
   'CantidadEncuestasCumplidas': 41},
  {'IdI

## **02. Lectura de todos los WebService**

In [10]:
# Diccionarios para almacenar los DataFrames por NIT
dataframes_instalaciones = {}
dataframes_rutas = {}

# Iterar sobre cada NIT en el DataFrame
for i in NITS['label']:
    # URL del web service con el NIT en la consulta
    url = f"https://webservices.metropol.gov.co/simapi/api/Terceros/PmesTercero?Doc={i}"

    # Realizar la solicitud al web service
    response = requests.get(url)
    
    if response.status_code == 200:
        data = response.json()
        instalaciones_data = []
        rutas_data = []

        # Procesar las instalaciones si existen
        if 'Instalaciones' in data and data['Instalaciones'] is not None:
            for instalacion in data['Instalaciones']:
                instalaciones_data.append({
                    'NIT': i,
                    'IdInstalacion': instalacion.get('IdInstalacion'),
                    'NombreInstalcion': instalacion.get('NombreInstalcion'),
                    'CantidadColaboradores': data.get('CantidadColaboradores'),
                    'CoordenadaX': instalacion.get('CoordenadaX'),
                    'CoordenadaY': instalacion.get('CoordenadaY'),
                    'CantEmpleados': instalacion.get('CantEmpleados'),
                    'CantidadEncuestasCumplidas': instalacion.get('CantidadEncuestasCumplidas'),
                    'CantInstalaciones' : data.get('CantInstalaciones'),
                    'CantRutas' : data.get('CantRutas')
                })

        # Procesar las rutas empresariales si existen
        if 'RutasEmpresariales' in data and data['RutasEmpresariales'] is not None:
            for ruta in data['RutasEmpresariales']:
                rutas_data.append({
                    'NIT': i,
                    'NombreRuta': ruta.get('NombreRuta'),
                    'Instalacion': ruta.get('Instalacion'),
                    'TipoVehiculo': ruta.get('TipoVehiculo'),
                    'TipoCombustible': ruta.get('TipoCombustible'),
                    'ModeloVehiculo': ruta.get('ModeloVehiculo'),
                    'CilindrajeVehiculo': ruta.get('CilindrajeVehiculo'),
                    'CantPromedioPasageros': ruta.get('CantPromedioPasageros')
                })

        # Crear un DataFrame para cada NIT y almacenarlo en los diccionarios
        if instalaciones_data:
            dataframes_instalaciones[i] = pd.DataFrame(instalaciones_data)
        if rutas_data:
            dataframes_rutas[i] = pd.DataFrame(rutas_data)
    else:
        print(f"Error al obtener datos para NIT {i}: {response.status_code}")

Error al obtener datos para NIT 9012494137: 400
Error al obtener datos para NIT 8909071635: 400
Error al obtener datos para NIT 8909047132: 400
Error al obtener datos para NIT 8909012983: 400
Error al obtener datos para NIT 8909064072: 400
Error al obtener datos para NIT 8909813747: 400
Error al obtener datos para NIT 8110046593: 400
Error al obtener datos para NIT 8909801366: 400
Error al obtener datos para NIT 8300806747: 400
Error al obtener datos para NIT 8909002918: 400
Error al obtener datos para NIT 8909029226: 400


In [9]:
data.get('CantidadColaboradores')

179

## **03. Generación de archivos csv para instalaciones y rutas extraidos del webservice**

In [11]:
import pandas as pd
import unidecode

# Función para limpiar nombres de columnas (remover acentos y reemplazar espacios por guiones bajos)
def limpiar_columnas(df):
    df.columns = [unidecode.unidecode(col).replace(" ", "_") for col in df.columns]
    return df

# Encontrar el número máximo de filas entre todos los dataframes
max_filas_instalaciones = max([len(df) for df in dataframes_instalaciones.values()])
max_filas_rutas = max([len(df) for df in dataframes_rutas.values()])

# Crear dataframes de instalaciones para cada fila y guardarlos como CSV
for i in range(max_filas_instalaciones):
    filas_instalaciones = []
    
    for nit, df_instalaciones in dataframes_instalaciones.items():
        if i < len(df_instalaciones):
            filas_instalaciones.append(df_instalaciones.iloc[i])
    
    # Crear un dataframe a partir de las filas de esta iteración
    df_instalaciones_i = pd.DataFrame(filas_instalaciones)

    # Agregar una columna de índice al principio
    df_instalaciones_i.insert(0, 'Index', range(0, len(df_instalaciones_i)))
    
    # Limpiar nombres de columnas
    cols_to_modify = ['NombreInstalcion']
    df_instalaciones_i[cols_to_modify] = df_instalaciones_i[cols_to_modify].apply(lambda x: x.str.replace(r'[\s-]', '_', regex=True))
    df_instalaciones_i = df_instalaciones_i.rename(columns={'Index': 'name', 'NIT': 'label'})
    
    # Limpiar contenido de columnas específicas
    for col in cols_to_modify:
        if col in df_instalaciones_i.columns:
            df_instalaciones_i[col] = df_instalaciones_i[col].apply(lambda x: unidecode.unidecode(str(x)).replace(" ", "_") if pd.notna(x) else x)

    # Dividir el dataframe en dos partes
    df_instalaciones_i_parte1 = df_instalaciones_i[['name', 'label', 'CantidadColaboradores', 'NombreInstalcion', 'CoordenadaX', 'CoordenadaY']]
    df_instalaciones_i_parte2 = df_instalaciones_i[['name', 'label', 'CantEmpleados', 'CantidadEncuestasCumplidas', 'CantInstalaciones', 'CantRutas']]

    merged_df_parte1 = pd.merge(NITS, df_instalaciones_i_parte1, on='label', how='right')
    columns_drop = {'EMPRESA', 'CANTIDAD_SEDES', 'COLABORADORES', 'name_y'}
    merged_df_parte1 = merged_df_parte1.drop(columns=columns_drop)
    columns_rename = {'name_x':'name'}
    merged_df_parte1 = merged_df_parte1.rename(columns=columns_rename)

    merged_df_parte2 = pd.merge(NITS, df_instalaciones_i_parte2, on='label', how='right')
    columns_drop = {'EMPRESA', 'CANTIDAD_SEDES', 'COLABORADORES', 'name_y'}
    merged_df_parte2 = merged_df_parte2.drop(columns=columns_drop)
    columns_rename = {'name_x':'name'}
    merged_df_parte2 = merged_df_parte2.rename(columns=columns_rename)

    # Guardar los dataframes divididos como archivos CSV
    merged_df_parte1.to_csv(f'instalaciones_parte1_fila_{i+1:02}.csv', index=False)
    merged_df_parte2.to_csv(f'instalaciones_parte2_fila_{i+1:02}.csv', index=False)


# Crear dataframes de rutas para cada fila, dividir en dos dataframes y guardarlos como CSV
for i in range(max_filas_rutas):
    filas_rutas = []
    
    for nit, df_rutas in dataframes_rutas.items():
        if i < len(df_rutas):
            filas_rutas.append(df_rutas.iloc[i])
    
    # Crear un dataframe a partir de las filas de esta iteración
    df_rutas_i = pd.DataFrame(filas_rutas)
    
    # Agregar una columna de índice al principio
    df_rutas_i.insert(0, 'Index', range(0, len(df_rutas_i)))

    # Limpiar nombres de columnas
    cols_to_modify = ['NombreRuta', 'Instalacion', 'TipoVehiculo']
    df_rutas_i[cols_to_modify] = df_rutas_i[cols_to_modify].apply(lambda x: x.str.replace(r'[\s-]', '_', regex=True))
    df_rutas_i = df_rutas_i.rename(columns={'Index': 'name', 'NIT': 'label'})

    # Limpiar contenido de columnas específicas
    for col in cols_to_modify:
        if col in df_rutas_i.columns:
            df_rutas_i[col] = df_rutas_i[col].apply(lambda x: unidecode.unidecode(str(x)).replace(" ", "_") if pd.notna(x) else x)
    
    
    # Dividir el dataframe en dos partes
    df_rutas_i_parte1 = df_rutas_i[['name', 'label', 'NombreRuta', 'Instalacion', 'TipoVehiculo']]
    df_rutas_i_parte2 = df_rutas_i[['name', 'label', 'ModeloVehiculo', 'CilindrajeVehiculo', 'TipoCombustible', 'CantPromedioPasageros']]
    
    merged_df_parte1 = pd.merge(NITS, df_rutas_i_parte1, on='label', how='left')
    columns_drop = {'CANTIDAD_SEDES', 'COLABORADORES', 'name_y'}
    merged_df_parte1 = merged_df_parte1.drop(columns=columns_drop)
    columns_rename = {'name_x':'name'}
    merged_df_parte1 = merged_df_parte1.rename(columns=columns_rename)

    merged_df_parte2 = pd.merge(NITS, df_rutas_i_parte2, on='label', how='left')
    columns_drop = {'CANTIDAD_SEDES', 'COLABORADORES', 'name_y'}
    merged_df_parte2 = merged_df_parte2.drop(columns=columns_drop)
    columns_rename = {'name_x':'name'}
    merged_df_parte2 = merged_df_parte2.rename(columns=columns_rename)

    # Guardar los dataframes divididos como archivos CSV
    merged_df_parte1.to_csv(f'rutas_parte1_fila_{i+1:02}.csv', index=False)
    merged_df_parte2.to_csv(f'rutas_parte2_fila_{i+1:02}.csv', index=False)

In [51]:
merged_df = pd.merge(NITS, df_rutas_i_parte2, on='label', how='left')
columns_drop = {'CANTIDAD_SEDES', 'COLABORADORES', 'name_y'}
merged_df = merged_df.drop(columns=columns_drop)
columns_rename = {'name_x':'name'}
merged_df = merged_df.rename(columns=columns_rename)
merged_df

,name,label,EMPRESA,ModeloVehiculo,CilindrajeVehiculo,TipoCombustible,CantPromedioPasageros
0,0,890907797,ALGAMAR_SA,NaN,NaN,NaN,NaN
1,1,890928257,ALICO_SAS,NaN,NaN,NaN,NaN
2,2,890904138,AMTEX_SAS,NaN,NaN,NaN,NaN
3,3,890913902,ANTIOQUENA_DE_AUTOMOTORES_Y_REPUESTOS_SA,NaN,NaN,NaN,NaN
4,4,890984423,Area_Metropolitana_del_Valle_de_Aburra,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
140,140,890984002,UNIVERSIDAD_CES,NaN,NaN,NaN,NaN
141,141,890980040,UNIVERSIDAD_DE_ANTIOQUIA,NaN,NaN,NaN,NaN
142,142,890901389,UNIVERSIDAD_EAFIT,NaN,NaN,NaN,NaN
143,143,8909029226,UNIVERSIDAD_PONTIFICIA_BOLIVARIANA,NaN,NaN,NaN,NaN
